In [1]:
import pandas as pd 
import numpy as np

In [2]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder

In [3]:
df = pd.read_csv('data/covid_toy.csv')
df.head()

,age,gender,fever,cough,city,has_covid
0,60,Male,103.0,Mild,Kolkata,No
1,27,Male,100.0,Mild,Delhi,Yes
2,42,Male,101.0,Mild,Delhi,No
3,31,Female,98.0,Mild,Kolkata,No
4,65,Female,101.0,Mild,Mumbai,No


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   age        100 non-null    int64  
 1   gender     100 non-null    object 
 2   fever      90 non-null     float64
 3   cough      100 non-null    object 
 4   city       100 non-null    object 
 5   has_covid  100 non-null    object 
dtypes: float64(1), int64(1), object(4)
memory usage: 4.8+ KB


In [6]:
df.isnull().sum()

age           0
gender        0
fever        10
cough         0
city          0
has_covid     0
dtype: int64

In [7]:
df.shape

(100, 6)

In [8]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(df.drop(columns=['has_covid']),df['has_covid'],
                                                test_size=0.2)

In [11]:
X_train

,age,gender,fever,cough,city
60,24,Female,102.0,Strong,Bangalore
79,48,Female,103.0,Mild,Kolkata
12,25,Female,99.0,Strong,Kolkata
61,81,Female,98.0,Strong,Mumbai
84,69,Female,98.0,Strong,Mumbai
...,...,...,...,...,...
24,13,Female,100.0,Strong,Kolkata
18,64,Female,98.0,Mild,Bangalore
34,74,Male,102.0,Mild,Mumbai
76,80,Male,100.0,Mild,Bangalore


## 1. Aam Zindagi

In [16]:
# adding simple imputer to fever col
si = SimpleImputer()
X_train_fever = si.fit_transform(X_train[['fever']])

# also the test data
X_test_fever = si.fit_transform(X_test[['fever']])
                                 
X_train_fever.shape

(80, 1)

In [29]:
X_train_fever

array([[102. ],
       [103. ],
       [ 99. ],
       [ 98. ],
       [ 98. ],
       [101. ],
       [100. ],
       [102. ],
       [100. ],
       [ 99. ],
       [100.9],
       [100.9],
       [ 99. ],
       [101. ],
       [100.9],
       [101. ],
       [ 98. ],
       [104. ],
       [100. ],
       [ 99. ],
       [104. ],
       [100. ],
       [100.9],
       [101. ],
       [104. ],
       [100.9],
       [ 98. ],
       [102. ],
       [100.9],
       [100. ],
       [ 99. ],
       [102. ],
       [ 98. ],
       [100.9],
       [101. ],
       [100. ],
       [101. ],
       [101. ],
       [102. ],
       [ 98. ],
       [ 99. ],
       [103. ],
       [ 98. ],
       [ 98. ],
       [104. ],
       [102. ],
       [104. ],
       [104. ],
       [100. ],
       [102. ],
       [ 98. ],
       [100. ],
       [101. ],
       [101. ],
       [100.9],
       [104. ],
       [ 98. ],
       [101. ],
       [ 99. ],
       [102. ],
       [104. ],
       [100.9],
       [

In [17]:
# Ordinalencoding -> cough
oe = OrdinalEncoder(categories=[['Mild','Strong']])
X_train_cough = oe.fit_transform(X_train[['cough']])

# also the test data
X_test_cough = oe.fit_transform(X_test[['cough']])

X_train_cough.shape

(80, 1)

In [30]:
X_train_cough

array([[1.],
       [0.],
       [1.],
       [1.],
       [1.],
       [0.],
       [1.],
       [0.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [1.],
       [0.],
       [1.],
       [0.],
       [0.],
       [1.],
       [1.],
       [1.],
       [0.],
       [0.],
       [0.],
       [1.],
       [1.],
       [0.],
       [1.],
       [1.],
       [0.],
       [1.],
       [1.],
       [1.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],

In [18]:
# OneHotEncoding -> gender,city
ohe = OneHotEncoder(drop='first',sparse_output=False)
X_train_gender_city = ohe.fit_transform(X_train[['gender','city']])

# also the test data
X_test_gender_city = ohe.fit_transform(X_test[['gender','city']])

X_train_gender_city.shape

(80, 4)

In [31]:
X_train_gender_city

array([[0., 0., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [1., 1., 0., 0.],
       [0., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 1.],
       [1., 0., 0., 0.],
       [0., 0., 0., 1.],
       [1., 0., 1., 0.],
       [0., 0., 0., 0.],
       [0., 0., 1., 0.],
       [1., 0., 0., 0.],
       [1., 0., 1., 0.],
       [0., 1., 0., 0.],
       [0., 0., 0., 0.],
       [1., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 1., 0.],
       [1., 0., 1., 0.],
       [0., 0., 0., 1.],
       [0., 1., 0., 0.],
       [1., 1., 0., 0.],
       [1., 0., 0., 1.],
       [0., 0., 1., 0.],
       [1., 1., 0., 0.],
       [0., 0., 1., 0.],
       [1., 0., 0., 0.],
       [1., 1., 0., 0.],
       [1., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],


In [20]:
# Extracting Age
X_train_age = X_train.drop(columns=['gender','fever','cough','city']).values

# also the test data
X_test_age = X_test.drop(columns=['gender','fever','cough','city']).values

X_train_age.shape

(80, 1)

In [21]:
X_train_transformed = np.concatenate((X_train_age,X_train_fever,X_train_gender_city,X_train_cough),axis=1)
# also the test data
X_test_transformed = np.concatenate((X_test_age,X_test_fever,X_test_gender_city,X_test_cough),axis=1)

X_train_transformed.shape

(80, 7)

## Mentos Zindagi


In [26]:
from sklearn.compose import ColumnTransformer

In [27]:
transformer = ColumnTransformer(transformers=[
    ('tnf1',SimpleImputer(),['fever']),
    ('tnf2',OrdinalEncoder(categories=[['Mild','Strong']]),['cough']),
    ('tnf3',OneHotEncoder(sparse_output=False,drop='first'),['gender','city'])
],remainder='passthrough')

In [28]:
transformer

ColumnTransformer(remainder='passthrough',
                  transformers=[('tnf1', SimpleImputer(), ['fever']),
                                ('tnf2',
                                 OrdinalEncoder(categories=[['Mild',
                                                             'Strong']]),
                                 ['cough']),
                                ('tnf3',
                                 OneHotEncoder(drop='first',
                                               sparse_output=False),
                                 ['gender', 'city'])])

In [17]:
transformer.fit_transform(X_train).shape

(80, 7)

In [18]:
transformer.transform(X_test).shape

(20, 7)